In [1]:
!pip install opendatasets

  Created wheel for kaggle: filename=kaggle-1.5.12-py3-none-any.whl size=73053 sha256=514e82bccb09ad66a25a018a02df371acbb91e3cd7ea61b35d18ad62cfb6026a
  Stored in directory: c:\users\donat\appdata\local\pip\cache\wheels\29\da\11\144cc25aebdaeb4931b231e25fd34b394e6a5725cbb2f50106
Successfully built kaggle


In [3]:
!pip install tensorflow

  Using cached opt_einsum-3.3.0-py3-none-any.whl (65 kB)
  Using cached Keras_Preprocessing-1.1.2-py2.py3-none-any.whl (42 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl (57 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl (12 kB)
  Using cached termcolor-1.1.0.tar.gz (3.9 kB)
  Using cached pyasn1_modules-0.2.8-py2.py3-none-any.whl (155 kB)
  Using cached requests_oauthlib-1.3.0-py2.py3-none-any.whl (23 kB)
  Using cached pyasn1-0.4.8-py2.py3-none-any.whl (77 kB)
  Created wheel for clang: filename=clang-5.0-py3-none-any.whl size=30702 sha256=cba4853eab105290088fff00de7b6e4c7f189b2d1e3e9094458f98170f6102df
  Stored in directory: c:\users\donat\appdata\local\pip\cache\wheels\f1\60\77\22b9b5887bd47801796a856f47650d9789c74dc3161a26d608
  Created wheel for termcolor: filename=termcolor-1.1.0-py3-none-any.whl size=4829 sha256=9aa1f355bbc18674d560a501e6ff2e800f6b0896e532c633acd35f327272bdda
  Stored in directory: c:\users\donat\appdata\local\pip\cache\wheels\a0\16\9c\5473df8

In [4]:
!pip install tensorflow_federated

  Using cached absl-py-0.9.0.tar.gz (104 kB)
  Using cached gast-0.3.3-py2.py3-none-any.whl (9.7 kB)
  Created wheel for absl-py: filename=absl_py-0.9.0-py3-none-any.whl size=121930 sha256=d48858ed8d8c44579afff41bc5db58d440a8598b2bf8ccf023c3260a634e1ea7
  Stored in directory: c:\users\donat\appdata\local\pip\cache\wheels\1d\10\8e\2f79b924179ff1e6510933d63eb851bea01054fff262343b7a
  Created wheel for retrying: filename=retrying-1.3.3-py3-none-any.whl size=11429 sha256=76388d1e8bf5d5b6e5964a04050c5aad93635b4b8aa1690deb4f6e60f34ed0c6
  Stored in directory: c:\users\donat\appdata\local\pip\cache\wheels\c4\a7\48\0a434133f6d56e878ca511c0e6c38326907c0792f67b476e56
Successfully built absl-py retrying
  Attempting uninstall: cachetools
    Found existing installation: cachetools 4.2.2
    Uninstalling cachetools-4.2.2:
      Successfully uninstalled cachetools-4.2.2
  Attempting uninstall: numpy
    Found existing installation: numpy 1.19.5
    Uninstalling numpy-1.19.5:
      Successfully unin

In [11]:
!pip install torch

In [13]:
!pip install torchvision

In [5]:
import opendatasets as od
from tensorflow import keras
import numpy as np

In [6]:
od.download("https://www.kaggle.com/prashant268/chest-xray-covid19-pneumonia")
od.download("https://www.kaggle.com/sudalairajkumar/novel-corona-virus-2019-dataset")

Skipping, found downloaded files in ".\chest-xray-covid19-pneumonia" (use force=True to force download)
Skipping, found downloaded files in ".\novel-corona-virus-2019-dataset" (use force=True to force download)


In [7]:
LEARNING_RATE = 0.001 # 0.0001
MAX_EPOCHS = 25
TARGET_FOLDER = "weights"

In [8]:
import torch
import torchvision
from torchvision import datasets, transforms

Resize Images to 244, 244. By using to_tensor the images are already normalized between 0 and 1. "The image object is an array of (244, 244, 3) should be flattened to be list (178, 608)." What? wie?

In [9]:
transform = transforms.Compose([transforms.Resize((244, 244))
                                , transforms.ToTensor()]
                               #, transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]) # find mean and std of dataset
                              )

test_set = datasets.ImageFolder('chest-xray-covid19-pneumonia/Data/test', transform=transform)

train_set = dataset = datasets.ImageFolder('chest-xray-covid19-pneumonia/Data/train', transform=transform)

In [10]:
def label_preparation(labels):
    labels = np.array(labels)
    labels[labels > 0] = 1
    return list(labels)

def label_preparation_tensor(labels):
    labels[labels > 0] = 1
    return labels

train_set.targets = label_preparation(train_set.targets)

test_set.targets = label_preparation(test_set.targets)

In [11]:
train_loader = torch.utils.data.DataLoader(train_set, batch_size=64, shuffle=True, num_workers=4)

test_loader = torch.utils.data.DataLoader(test_set, batch_size=64, shuffle=False, num_workers=4)

In Paper aus References haben sie unter anderen ResNet18 benutzt: https://arxiv.org/pdf/2007.05592.pdf

Deshalb würde ich das probieren.

TODO: test different ones, e.g. VGG19

In [12]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

print(f"Current device: {device}")

Current device: cuda:0


In [13]:
def calc_accuracy(result, labels):
    result = torch.sigmoid(result).round()
    
    correct_results_sum = (result == labels).sum().float()
    acc = correct_results_sum/labels.shape[0]
    acc *= 100
    
    return acc


In [14]:
def train(model, data_loader, optimizer, loss):
    """
    model -- neural net
    data_loader -- dataloader for train images
    optimizer -- optimizer
    """
    model.train()
    
    # use pos weights because of unbalanced data set
    accuracy = 0
    for step, [images, labels] in enumerate(data_loader, 1):
        images = images.to(device)
        labels = label_preparation_tensor(labels.to(device))

        optimizer.zero_grad()
        
        result = model(images)
        targets = labels.unsqueeze(1).float()

        loss_value = loss(result.float(), targets)

        # backpropagation
        loss_value.backward()
        optimizer.step()
                                    
        if step % 10 == 0:
            accuracy += calc_accuracy(result, labels.unsqueeze(1))
            print(f"TRAINING - Step: {step}, loss: {loss_value}, rolling accuracy: {accuracy*10/step}")

In [15]:
def test(model, test_loader, loss):
    """    
    model -- neural net 
    test_loader -- dataloader of test images
    epoch -- current epoch
    """
    model.eval()
    
    with torch.no_grad():
        loss_value = 0
        accuracy = 0
        for step, [images, labels] in enumerate(test_loader, 1):
            images = images.to(device)
            labels = label_preparation_tensor(labels.to(device))

            result = model(images)
            targets = labels.detach().unsqueeze(1).float()

            loss_value += loss(result.detach(), targets)
            accuracy += calc_accuracy(result.detach(), labels.detach().unsqueeze(1))

        loss_value /= step
        accuracy /=  step
  
    print(f"TESTING - Loss: {loss_value}, Accuracy: {accuracy}")
    return accuracy > 93.

In [16]:
def run_local_training():
    model = torchvision.models.resnet18(pretrained=False, num_classes=1)
    model.to(device)

    # initialize optimizer
    optimizer = torch.optim.Adam(model.parameters())

    # use pos weights because of unbalanced data set
    loss = torch.nn.BCEWithLogitsLoss(pos_weight=torch.tensor([1./10])).to(device) # binary crossentropy

    # start training
    for epoch in range(MAX_EPOCHS):
        print(f"+++ EPOCH: {epoch+1} +++++++++")
        torch.nn.BCEWithLogitsLoss(pos_weight=torch.tensor([1./10])).to(device) # binary crossentropy (non-federated)
        train(model, train_loader, optimizer, loss)
        break
        # save interim weights
        torch.save(model.state_dict(), f'./{TARGET_FOLDER}/epoch_{epoch}.ckpt')

        if test(model, test_loader, loss) and epoch > 4:
            break

In [17]:
#run_local_training()

# Federated
https://blog.openmined.org/upgrade-to-federated-learning-in-10-lines/

i don't think we need syft if we just simulate the federation on one machine

In [18]:
from collections import OrderedDict

In [19]:
number_of_clients = 4

In [20]:
federated_model = torchvision.models.resnet18(pretrained=False, num_classes=1)
federated_model.to(device)

def reset_client_param_list():
    return [None] * number_of_clients
    
client_params = reset_client_param_list()

In [30]:
client_models = [torchvision.models.resnet18(pretrained=False, num_classes=1).to(device) for _ in range(number_of_clients)]
client_optimizers = [torch.optim.Adam(client_models[idx].parameters()) for idx in range(number_of_clients)]
client_training_loader = [torch.utils.data.DataLoader(train_set, batch_size=64, shuffle=True, num_workers=4) for _ in range(number_of_clients)]

In [31]:
def send_params_to_global_model(client_params, client_idx, params):
    client_params[client_idx] = params
    for idx in range (number_of_clients):
        if client_params[idx] is None:
            return client_params, False
        
    return client_params, True

def send_params_to_client(federated_model, client):
    client.load_state_dict(federated_model.state_dict(), True)
    return client
    
def federated_average(federated_model, client_params):
    with torch.no_grad():
        average_weights = OrderedDict()

        # check if client_params has no Nones?
        for client_param in client_params:
            for key, value in client_param.items():
                if key in average_weights:
                    average_weights[key] += value.clone()
                else:
                    average_weights[key] = value.clone()

        for key, value in average_weights.items():
            average_weights[key] = torch.div(average_weights[key], len(client_params), rounding_mode='floor')
                
                
    federated_model.load_state_dict(average_weights, True)
    return federated_model
    

In [32]:
def train_federated(model, data_loader, optimizer, loss):
    """
    model -- neural net
    data_loader -- dataloader for train images
    optimizer -- optimizer
    """
    model.train()
    
    # use pos weights because of unbalanced data set
    accuracy = 0
    for step, [images, labels] in enumerate(data_loader, 1):
        images = images.to(device)
        labels = label_preparation_tensor(labels.to(device))

        optimizer.zero_grad()
        
        result = model(images)
        targets = labels.unsqueeze(1).float()

        loss_value = loss(result.float(), targets)

        # backpropagation
        loss_value.backward()
        optimizer.step()
                                    
        if step % 10 == 0:
            accuracy += calc_accuracy(result, labels.unsqueeze(1))
            print(f"TRAINING - Step: {step}, loss: {loss_value}, rolling accuracy: {accuracy*10/step}")

In [35]:
def test_federated(model, test_loader, loss):
    """    
    model -- neural net 
    test_loader -- dataloader of test images
    epoch -- current epoch
    """
    model.eval()
    
    # use pos weights because of unbalanced data set
    criterion = torch.nn.BCEWithLogitsLoss(pos_weight=torch.tensor([1./10])).to(device) # binary crossentropy (non-federated)
    
    with torch.no_grad():
        loss_value = 0
        accuracy = 0
        for step, [images, labels] in enumerate(test_loader, 1):
            images = images.to(device)
            labels = label_preparation_tensor(labels.to(device))

            result = model(images)
            targets = labels.detach().unsqueeze(1).float()

            loss_value += loss(result.detach(), targets)
            accuracy += calc_accuracy(result.detach(), labels.detach().unsqueeze(1))

        loss_value /= step
        accuracy /=  step
  
    print(f"TESTING - Loss: {loss_value}, Accuracy: {accuracy}")
    return accuracy > 93.

In [37]:
def run_federated_training(federated_model, client_models, client_optimizers, client_params, client_training_loader):
    # use pos weights because of unbalanced data set
    federated_loss = torch.nn.BCEWithLogitsLoss(pos_weight=torch.tensor([1./10])).to(device) # binary crossentropy
    # federated_loss = torch.nn.CrossEntropyLoss(weight=torch.tensor([1./10])).to(device) # sparce categorical crossentropy (federated)

    # start training
    for epoch in range(MAX_EPOCHS):
        for client_idx in range (number_of_clients):
            print(f"+++ FEDERATED MODEL {client_idx}, EPOCH: {epoch+1} +++++++++")

            client_model = client_models[client_idx]
            client_model = send_params_to_client(federated_model, client_model)

            client_optimizer = client_optimizers[client_idx]

            train_federated(client_model, client_training_loader[client_idx], client_optimizer, federated_loss)

            client_params, _ = send_params_to_global_model(client_params, client_idx, client_model.state_dict())


            # save interim weights
            #torch.save(model.state_dict(), f'./{TARGET_FOLDER}/epoch_{epoch}.ckpt')


        federated_model = federated_average(federated_model, client_params)
        client_params = reset_client_param_list()
        
        test_federated(federated_model, test_loader, federated_loss)


In [ ]:
run_federated_training(federated_model, client_models, client_optimizers, client_params, client_training_loader)

+++ FEDERATED MODEL 0, EPOCH: 1 +++++++++


C:\Users\Donat\Anaconda3\envs\Python_38\lib\site-packages\torch\nn\functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ..\c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


TRAINING - Step: 10, loss: 0.04027961939573288, rolling accuracy: 89.0625
TRAINING - Step: 20, loss: 0.015832576900720596, rolling accuracy: 90.625
TRAINING - Step: 30, loss: 0.10454931855201721, rolling accuracy: 92.18750762939453
TRAINING - Step: 40, loss: 0.05478958040475845, rolling accuracy: 89.84375
TRAINING - Step: 50, loss: 0.05593022704124451, rolling accuracy: 91.25
TRAINING - Step: 60, loss: 0.055436667054891586, rolling accuracy: 90.88542175292969
TRAINING - Step: 70, loss: 0.009545935317873955, rolling accuracy: 91.74107360839844
TRAINING - Step: 80, loss: 0.017689980566501617, rolling accuracy: 92.1875
+++ FEDERATED MODEL 1, EPOCH: 1 +++++++++
TRAINING - Step: 10, loss: 0.03219511732459068, rolling accuracy: 93.75
TRAINING - Step: 20, loss: 0.057915907353162766, rolling accuracy: 91.40625
TRAINING - Step: 30, loss: 0.026234066113829613, rolling accuracy: 90.62500762939453
TRAINING - Step: 40, loss: 0.010355543345212936, rolling accuracy: 91.796875
TRAINING - Step: 50, los